In [2]:
import pandas as pd
import h5py
import numpy as np
from pathlib import Path

def cal_snr(data):
    # input a 3C signal and cal the snr, the first 50 sample are noise
    noise = data[0, :50] 
    signal = data[0, 50:100] 
    signal_energy_mean = np.sum(signal**2) 
    noise_energy_mean = np.sum(noise**2) 
    snr = 10 * np.log10(signal_energy_mean / noise_energy_mean)
    return snr

def checkdata(data):
    return all([d.any() for d in data])

raw_root = Path('E:/_datasets/STEAD/raw') # download the raw data at https://github.com/smousavi05/STEAD

# row/
#     chunk2/
#         chunk2.csv
#         chunk2.hdf5
#     chunk3/
#         chunk3.csv
#         chunk3.hdf5
#     chunk4/
#         chunk4.csv
#         chunk4.hdf5
#     chunk5/
#         chunk5.csv
#         chunk5.hdf5
#     chunk6/
#         chunk6.csv
#         chunk6.hdf5


In [3]:
meta_path = Path('meta.csv')
if not meta_path.exists():
    meta = pd.DataFrame()
    for i in [2, 3, 4, 5, 6]:
        meta_i = pd.read_csv(raw_root/f"chunk{i}/chunk{i}.csv", low_memory=False)
        # filter the data
        meta_i = meta_i.loc[meta_i.eval(
            "(p_arrival_sample > 50) and \
            (s_arrival_sample - p_arrival_sample > 250) and \
            (receiver_type=='HH') and\
            (source_distance_deg<=1)"), :].reset_index(drop=True)
        selected = []
        with h5py.File(raw_root/f'chunk{i}/chunk{i}.hdf5') as h5:
            with h5py.File("data.hdf5", 'a') as h5_new:
                for j, (name, spt) in enumerate(zip(meta_i.trace_name, meta_i.p_arrival_sample)):
                    d = np.array(h5.get(f'data/{name}')).T[:,int(spt)-50:int(spt)+250]
                    # snr > 10 db
                    if checkdata(d) and cal_snr(d) > 10:    
                        if not h5_new.get(f'data/{name}'):
                            h5_new.create_dataset(f'data/{name}', data=d)
                            selected.append(j)
        meta = pd.concat([meta, meta_i.iloc[selected, :]]).reset_index(drop=True)  
        print(meta.shape)  
        meta.to_csv(meta_path, index=False, sep='\t')

In [4]:
# split tran/test set
meta = pd.read_csv('meta.csv', sep='\t', low_memory=False)
if 'is_train' not in meta.columns:
    from sklearn.model_selection import train_test_split
    train_index, test_index =  train_test_split(np.arange(meta.shape[0]), test_size=0.2)
    is_train = np.zeros(meta.shape[0], dtype=bool)
    is_train[train_index] = True
    meta['is_train'] = is_train
    meta.to_csv('meta.csv', index=False, sep='\t')

In [5]:
# calculate the azimuths
meta = pd.read_csv('meta.csv', sep='\t', low_memory=False)
if 'azimuth' not in meta.columns:
    import obspy
    def cal_azimuth(row):
        return obspy.geodetics.base.gps2dist_azimuth(
            row['receiver_latitude'],
            row['receiver_longitude'],
            row['source_latitude'],
            row['source_longitude'],
            a=6378137.0, 
            f=0.0033528106647474805)[1]
    meta['azimuth'] = meta.apply(cal_azimuth, axis=1)
    meta.to_csv('meta.csv', index=False, sep='\t')